# Подготовка ноутбука 

## Пробрасываем magic methods

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Imports

In [2]:
import pandas as pd 
import numpy as np 
from data.prepare_data import PrepareData
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from torch.utils.data import DataLoader
from IPython.display import display
import pytesseract
import shutil
try:
    from PIL import Image
except ImportError:
     import Image
import cv2

## Нужные переменные 

In [3]:
ROOT_DIR = Path('../../')
load_dotenv()
filterwarnings(action='ignore')

# Загрузка датафрейма

# Тут я просто делю на трейн и тест по своему имно из INC

In [3]:
# pip install pandas pyarrow
import os
import pandas as pd
from sklearn.model_selection import train_test_split

EXCEL_PATH = r"/home/lanmo/hack_digital_transformation/data/raw_data/data/metadata/INC/18-001_gin_building_echd_19.08.25.xlsx"
IMAGES_ROOT = r"/home/lanmo/hack_digital_transformation/data/raw_data/data/metadata/INC/18-001_gin_building_echd_19.08.25"
CSV_ALL   = r"/home/lanmo/hack_digital_transformation/notebooks/all_images.csv"
CSV_TRAIN = r"/home/lanmo/hack_digital_transformation/notebooks/train.csv"
CSV_VAL   = r"/home/lanmo/hack_digital_transformation/notebooks/val.csv"

def excel_to_csv(excel_path, images_root, csv_out,
                 filename_col="Имя файла",
                 lat_col="latitude", lon_col="longitude",
                 n_subset=None, val_size=0.1, seed=42):
    # читаем как строки, чтобы не сломать числа с запятой
    df = pd.read_excel(excel_path, dtype=str)

    # переименуем/нормализуем нужные колонки
    rename_map = {
        filename_col: "path",
        lat_col: "lat",
        lon_col: "lon",
    }
    for k in [filename_col, lat_col, lon_col]:
        if k not in df.columns:
            raise ValueError(f"В Excel нет обязательной колонки: {k}")

    df = df.rename(columns=rename_map)[["path", "lat", "lon"]]

    # абсолютные пути к картинкам
    df["path"] = df["path"].apply(lambda x: os.path.join(images_root, str(x)))

    # lat/lon: запятая -> точка -> float
    df["lat"] = df["lat"].str.replace(",", ".", regex=False).astype(float)
    df["lon"] = df["lon"].str.replace(",", ".", regex=False).astype(float)

    # оставляем только существующие файлы
    df = df[df["path"].apply(os.path.exists)].reset_index(drop=True)

    # опционально берём подмножество
    if n_subset is not None and n_subset < len(df):
        df = df.sample(n_subset, random_state=seed).reset_index(drop=True)

    # сохраняем общий CSV
    df.to_csv(csv_out, index=False)
    print(f"Сохранён CSV: {csv_out} (записей: {len(df)})")

    # train/val
    train_df, val_df = train_test_split(df, test_size=val_size, random_state=seed)
    train_df.to_csv(CSV_TRAIN, index=False)
    val_df.to_csv(CSV_VAL, index=False)
    print(f"train: {len(train_df)}  val: {len(val_df)}")

# пример вызова: возьмём 5000 строк для прогона
excel_to_csv(EXCEL_PATH, IMAGES_ROOT, CSV_ALL, n_subset=30000)



Сохранён CSV: /home/lanmo/hack_digital_transformation/notebooks/all_images.csv (записей: 7707)
train: 6936  val: 771


# Очень годно детектит

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import torch
from transformers import Owlv2Processor, Owlv2ForObjectDetection  

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Owlv2Processor.from_pretrained("google/owlv2-base-patch16-ensemble")
model = Owlv2ForObjectDetection.from_pretrained("google/owlv2-base-patch16-ensemble").to(device).eval()

texts = [["house", "building", "apartment building", "home"]]

def detect_houses(image_path, score_thresh=0.25, iou_thresh=0.5, out_path="pred.jpg"):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=texts, images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]]).to(device)  # (H, W)
    results = processor.post_process_object_detection(
        outputs=outputs,
        threshold=score_thresh,
        target_sizes=target_sizes
    )[0]

    boxes = results["boxes"].cpu()    
    scores = results["scores"].cpu()
    labels = results["labels"].cpu()  

    keep = torch.ops.torchvision.nms(boxes, scores, iou_thresh)
    boxes, scores, labels = boxes[keep], scores[keep], labels[keep]


    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 18)
    except:
        font = None

    for box, score, lab in zip(boxes, scores, labels):
        x1, y1, x2, y2 = map(float, box.tolist())
        draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=3)
        name = texts[0][int(lab)]
        txt = f"{name}: {score:.2f}"
        tw, th = draw.textlength(txt, font=font), 18
        draw.rectangle([x1, y1 - th - 4, x1 + tw + 6, y1], fill=(0, 255, 0))
        draw.text((x1 + 3, y1 - th - 2), txt, fill=(0, 0, 0), font=font)

    image.save(out_path)
    print(f"Готово: найдено {len(boxes)} боксов. Сохранено в {out_path}")
    return boxes.numpy(), scores.numpy()

detect_houses(тут ссылка, score_thresh=0.25, iou_thresh=0.5, out_path="street_houses.jpg")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Готово: найдено 3 боксов. Сохранено в street_houses.jpg


/home/lanmo/hack_digital_transformation/.venv/lib/python3.12/site-packages/transformers/models/owlv2/processing_owlv2.py:201: FutureWarning: `post_process_object_detection` method is deprecated for OwlVitProcessor and will be removed in v5. Use `post_process_grounded_object_detection` instead.
  warnings.warn(


(array([[4.1678829e+01, 6.5760934e-01, 7.0213861e+02, 4.8493274e+02],
        [7.2962201e+02, 3.9257327e+02, 8.5870062e+02, 4.8749780e+02],
        [1.1606184e+00, 2.3607016e-02, 7.7130653e+01, 4.6877594e+02]],
       dtype=float32),
 array([0.7730474 , 0.42973122, 0.39342672], dtype=float32))

In [2]:

import os
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import faiss
from geoclip import ImageEncoder


# ------------------------- Data utils -------------------------

@dataclass
class Dataset:
    paths: List[str]
    lat: Optional[np.ndarray] = None
    lon: Optional[np.ndarray] = None
    label: Optional[np.ndarray] = None


def read_csv_dataset(csv_path: str,
                     path_col: str = "path",
                     lat_col: str = "lat",
                     lon_col: str = "lon",
                     label_col: str = "label") -> Dataset:
    df = pd.read_csv(csv_path)
    if path_col not in df.columns:
        raise ValueError(f"CSV {csv_path} не содержит столбец '{path_col}'.")
    paths = df[path_col].astype(str).tolist()
    lat = df[lat_col].to_numpy() if lat_col in df.columns else None
    lon = df[lon_col].to_numpy() if lon_col in df.columns else None
    label = df[label_col].to_numpy() if label_col in df.columns else None
    return Dataset(paths=paths, lat=lat, lon=lon, label=label)


def load_image(path: str) -> Image.Image:
    return Image.open(path).convert("RGB")


# ------------------------- Encoding -------------------------

def get_model_device(model: torch.nn.Module) -> torch.device:
    try:
        return next(model.parameters()).device
    except Exception:
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")


def encode_images(
    image_encoder: ImageEncoder,
    image_paths: List[str],
    batch_size: int = 64,
    desc: str = "Encoding",
) -> np.ndarray:
    """
    Кодирует список путей в эмбеддинги, используя строго API geoclip:
      - image_encoder.preprocess_image(PIL.Image) -> torch.Tensor [C,H,W] или [1,C,H,W]
      - image_encoder(tensor[B,C,H,W]) -> torch.Tensor [B,D]
    Работает батчами и показывает прогресс через tqdm.
    """
    device = get_model_device(image_encoder)
    image_encoder.eval()  # на всякий случай
    embs_chunks: List[np.ndarray] = []

    cur_batch: List[torch.Tensor] = []
    for path in tqdm(image_paths, desc=f"{desc}: load+preprocess", unit="img"):
        img = load_image(path)
        t = image_encoder.preprocess_image(img)  # -> torch.Tensor
        if not isinstance(t, torch.Tensor):
            t = torch.as_tensor(t)

        if t.ndim == 3:  # [C,H,W] -> [1,C,H,W]
            t = t.unsqueeze(0)

        cur_batch.append(t)

        if len(cur_batch) >= batch_size:
            batch = torch.cat(cur_batch, dim=0).to(device)  # [B,C,H,W]
            with torch.no_grad():
                out = image_encoder(batch)                  # [B,D]
            out = out.detach().cpu().numpy().astype("float32", copy=False)
            embs_chunks.append(out)
            cur_batch.clear()

    # хвост
    if cur_batch:
        batch = torch.cat(cur_batch, dim=0).to(device)
        with torch.no_grad():
            out = image_encoder(batch)
        out = out.detach().cpu().numpy().astype("float32", copy=False)
        embs_chunks.append(out)

    return np.vstack(embs_chunks)


def l2_normalize(x: np.ndarray, axis: int = 1, eps: float = 1e-12) -> np.ndarray:
    norm = np.linalg.norm(x, ord=2, axis=axis, keepdims=True)
    return x / np.maximum(norm, eps)


# ------------------------- FAISS -------------------------

def build_faiss_ip_index(vectors: np.ndarray) -> faiss.Index:
    """
    Индекс для косинусной близости:
    при L2-нормировке векторов косинусная схожесть == скалярному произведению.
    """
    d = vectors.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(vectors)
    return index


def search_knn_batched(
    index: faiss.Index,
    query_vectors: np.ndarray,
    k: int,
    search_batch_size: int = 4096,
    desc: str = "FAISS search",
) -> Tuple[np.ndarray, np.ndarray]:
    all_scores, all_indices = [], []
    for i in tqdm(range(0, len(query_vectors), search_batch_size), desc=desc, unit="batch"):
        q = query_vectors[i:i+search_batch_size]
        s, idx = index.search(q, k)
        all_scores.append(s)
        all_indices.append(idx)
    return np.vstack(all_scores), np.vstack(all_indices)


# ------------------------- Metrics -------------------------

def haversine_m(c1: Tuple[float, float], c2: Tuple[float, float]) -> float:
    """Расстояние между двумя точками (lat, lon) в метрах."""
    R = 6371000.0
    lat1, lon1 = np.radians(c1[0]), np.radians(c1[1])
    lat2, lon2 = np.radians(c2[0]), np.radians(c2[1])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return float(2 * R * np.arcsin(np.sqrt(a)))


def compute_geo_metrics(
    results_df: pd.DataFrame,
    ks: List[int] = [1, 5, 10],
    radii_m: List[float] = [25.0, 50.0, 100.0],
) -> Dict[str, float]:
    """Считает Geo-Recall@K и Geo-MRR@K по спискам K и радиусов; также среднюю топ-1 дистанцию."""
    metrics: Dict[str, float] = {}

    # Проверим, есть ли координаты
    if not {"test_lat", "test_lon", "train_lat", "train_lon"}.issubset(results_df.columns):
        return metrics  # нет координат — нет гео-метрик

    # Средняя дистанция до top-1 соседа
    top1_rows = results_df[results_df["neighbor_rank"] == 1].copy()
    valid_top1 = top1_rows.dropna(subset=["test_lat", "test_lon", "train_lat", "train_lon"])
    if len(valid_top1) > 0:
        dists = [
            haversine_m((r["test_lat"], r["test_lon"]), (r["train_lat"], r["train_lon"]))
            for _, r in valid_top1.iterrows()
        ]
        metrics["Top1_Distance_mean_m"] = float(np.mean(dists))
        metrics["Top1_Distance_median_m"] = float(np.median(dists))
        metrics["Top1_Distance_std_m"] = float(np.std(dists))
        metrics["Coverage_with_coords_%"] = 100.0 * len(valid_top1) / top1_rows.shape[0]

    # Recall@K и MRR@K для разных радиусов
    grouped = results_df.groupby("test_id")

    for rad in radii_m:
        for K in ks:
            hits = []
            rr_list = []
            for test_id, group in grouped:
                groupK = group.nsmallest(K, "neighbor_rank")
                groupK = groupK.dropna(subset=["test_lat", "test_lon", "train_lat", "train_lon"])
                if groupK.empty:
                    hits.append(False)
                    rr_list.append(0.0)
                    continue

                # Найдём ранг первого соседа в радиусе
                found_rank = None
                for _, row in groupK.iterrows():
                    d = haversine_m((row["test_lat"], row["test_lon"]),
                                    (row["train_lat"], row["train_lon"]))
                    if d <= rad:
                        found_rank = int(row["neighbor_rank"])
                        break

                if found_rank is None:
                    hits.append(False)
                    rr_list.append(0.0)
                else:
                    hits.append(True)
                    rr_list.append(1.0 / found_rank)

            metrics[f"Geo_Recall@{K}_≤{int(rad)}m"] = float(np.mean(hits)) if hits else np.nan
            metrics[f"Geo_MRR@{K}_≤{int(rad)}m"] = float(np.mean(rr_list)) if rr_list else np.nan

    return metrics


def compute_label_recall_at_k(
    results_df: pd.DataFrame,
    ks: List[int] = [1, 5, 10],
    label_col: str = "label",
) -> Dict[str, float]:
    """Если в данных есть 'label', считаем Recall@K по лейблам (правильный — сосед с тем же label)."""
    metrics: Dict[str, float] = {}
    need_cols = {"test_id", "neighbor_rank", label_col}
    if not need_cols.issubset(results_df.columns):
        return metrics
    grouped = results_df.groupby("test_id")
    for K in ks:
        hits = []
        for test_id, group in grouped:
            # test label предполагаем одинаковый в пределах группы
            if group[label_col].isna().all():
                hits.append(False)
                continue
            test_label = group[label_col].iloc[0]
            topK = group.nsmallest(K, "neighbor_rank")
            ok = (topK[label_col] == test_label).any()
            hits.append(bool(ok))
        metrics[f"Label_Recall@{K}"] = float(np.mean(hits)) if hits else np.nan
    return metrics


# ------------------------- Pipeline -------------------------

def knn_on_train_test(
    train_csv: str,
    test_csv: str,
    k: int = 5,
    batch_size: int = 64,
    search_batch_size: int = 4096,
    normalize: bool = True,
    save_index_path: Optional[str] = None,
    save_embeddings_dir: Optional[str] = None,
    path_col: str = "path",
    lat_col: str = "lat",
    lon_col: str = "lon",
    label_col: str = "label",
    metric_ks: List[int] = [1, 5, 10],
    metric_radii_m: List[float] = [25.0, 50.0, 100.0],
) -> Tuple[pd.DataFrame, Dict[str, float]]:
    # 1) Данные
    train_ds = read_csv_dataset(train_csv, path_col, lat_col, lon_col, label_col)
    test_ds  = read_csv_dataset(test_csv,  path_col, lat_col, lon_col, label_col)

    # 2) Энкодер
    image_encoder = ImageEncoder()

    # 3) Эмбеддинги (с прогрессом)
    train_embs = encode_images(image_encoder, train_ds.paths, batch_size=batch_size, desc="Train enc")
    test_embs  = encode_images(image_encoder, test_ds.paths,  batch_size=batch_size, desc="Test enc")

    # 4) Нормализация (для косинусной близости через IP)
    if normalize:
        train_embs = l2_normalize(train_embs)
        test_embs  = l2_normalize(test_embs)

    # (опционально) Сохранить эмбеддинги
    if save_embeddings_dir:
        os.makedirs(save_embeddings_dir, exist_ok=True)
        np.save(os.path.join(save_embeddings_dir, "train_embs.npy"), train_embs)
        np.save(os.path.join(save_embeddings_dir, "test_embs.npy"),  test_embs)

    # 5) Индекс FAISS
    index = build_faiss_ip_index(train_embs)
    if save_index_path:
        faiss.write_index(index, save_index_path)

    # 6) Поиск kNN (с прогрессом)
    scores, indices = search_knn_batched(index, test_embs, k=k, search_batch_size=search_batch_size)

    # 7) Сбор результатов (с прогрессом)
    rows = []
    for i in tqdm(range(len(test_ds.paths)), desc="Assembling results", unit="query"):
        test_path = test_ds.paths[i]
        test_lat  = None if test_ds.lat is None else float(test_ds.lat[i])
        test_lon  = None if test_ds.lon is None else float(test_ds.lon[i])
        test_lbl  = None if test_ds.label is None else test_ds.label[i]

        for j in range(k):
            train_idx = int(indices[i, j])
            row = {
                "test_id": i,
                "test_path": test_path,
                "neighbor_rank": j + 1,
                "train_id": train_idx,
                "train_path": train_ds.paths[train_idx],
                "similarity": float(scores[i, j]),
                "test_lat":  test_lat,
                "test_lon":  test_lon,
                "train_lat": None if train_ds.lat is None else float(train_ds.lat[train_idx]),
                "train_lon": None if train_ds.lon is None else float(train_ds.lon[train_idx]),
            }
            # Прокинем label, если есть
            if test_ds.label is not None:
                row["label"] = test_lbl
                if train_ds.label is not None:
                    row["train_label"] = train_ds.label[train_idx]
            rows.append(row)

    result_df = pd.DataFrame(rows)

    # 8) Метрики
    metrics = {}
    metrics.update(compute_geo_metrics(result_df, ks=metric_ks, radii_m=metric_radii_m))
    # Label-based только если есть колонка label и (опц.) train_label
    if "label" in result_df.columns:
        # Для совместимости: если нет train_label, используем label из train при сборке выше
        if "train_label" in result_df.columns:
            # подменим столбец для проверки совпадения
            same = (result_df["train_label"] == result_df["label"])
            result_df = result_df.assign(_same_label=same)
        metrics.update(compute_label_recall_at_k(result_df, ks=metric_ks, label_col="label"))

    return result_df, metrics


# ------------------------- Entry point -------------------------

if __name__ == "__main__":
    # Укажите свои CSV
    TRAIN_CSV = r"/home/lanmo/hack_digital_transformation/notebooks/train.csv"   # CSV со столбцами path,lat,lon,[label]
    TEST_CSV  = r"/home/lanmo/hack_digital_transformation/notebooks/val.csv"    # CSV со столбцами path,lat,lon,[label]

    # Гиперпараметры
    K = 5
    BATCH = 128
    SEARCH_BATCH = 512  # уменьшите, если не хватает RAM

    # Наборы K и радиусов (м) для метрик
    METRIC_KS = [1, 5, 10]
    METRIC_RADII = [25.0, 50.0, 100.0]

    results_df, metrics = knn_on_train_test(
        train_csv=TRAIN_CSV,
        test_csv=TEST_CSV,
        k=K,
        batch_size=BATCH,
        search_batch_size=SEARCH_BATCH,
        normalize=True,
        save_index_path="faiss_index.ip",        # или None
        save_embeddings_dir="embeddings_cache",  # или None
        metric_ks=METRIC_KS,
        metric_radii_m=METRIC_RADII,
    )

    out_csv = "knn_results.csv"
    results_df.to_csv(out_csv, index=False)
    print(f"\nГотово! Результаты сохранены в {out_csv}")

    # Печать метрик красиво
    if metrics:
        print("\n=== Metrics ===")
        # Сортируем ключи для аккуратного вывода
        for k in sorted(metrics.keys()):
            v = metrics[k]
            if "Recall" in k or "MRR" in k:
                print(f"{k:>24}: {v:.4f}")
            elif "Distance" in k or "Coverage" in k:
                print(f"{k:>24}: {v:.2f}")
            else:
                try:
                    print(f"{k:>24}: {float(v):.4f}")
                except Exception:
                    print(f"{k:>24}: {v}")
    else:
        print("\n(Координаты/лейблы не найдены — метрики не посчитаны.)")

Assembling results: 100%|██████████| 771/771 [00:00<00:00, 67962.85query/s]



Готово! Результаты сохранены в knn_results.csv

=== Metrics ===
  Coverage_with_coords_%: 100.00
        Geo_MRR@10_≤100m: 0.1944
         Geo_MRR@10_≤25m: 0.1768
         Geo_MRR@10_≤50m: 0.1845
         Geo_MRR@1_≤100m: 0.1751
          Geo_MRR@1_≤25m: 0.1634
          Geo_MRR@1_≤50m: 0.1699
         Geo_MRR@5_≤100m: 0.1944
          Geo_MRR@5_≤25m: 0.1768
          Geo_MRR@5_≤50m: 0.1845
     Geo_Recall@10_≤100m: 0.2283
      Geo_Recall@10_≤25m: 0.2023
      Geo_Recall@10_≤50m: 0.2114
      Geo_Recall@1_≤100m: 0.1751
       Geo_Recall@1_≤25m: 0.1634
       Geo_Recall@1_≤50m: 0.1699
      Geo_Recall@5_≤100m: 0.2283
       Geo_Recall@5_≤25m: 0.2023
       Geo_Recall@5_≤50m: 0.2114
    Top1_Distance_mean_m: 12713.91
  Top1_Distance_median_m: 11791.44
     Top1_Distance_std_m: 10583.09
